In [2]:
import os
import xarray as xr
import numpy as np
from netCDF4 import Dataset
from collections import defaultdict
from wrf import getvar, ALL_TIMES, interplevel, to_np, latlon_coords, get_cartopy, extract_times, ll_to_xy, to_np
from functions.listwrfouts import listWrfouts
from functions.injectmissincoords import inject_missing_coordinates_from_geo_em
from functions.plots import plot_T2_15facet, plot_PSFC_15facet, plot_TP_15facet, plot_WS10_15facet, plot_RH2_15facet
from functions.get_point_data import get_point_data
from functools import partial
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cartopy.crs as ccrs
import gc
import pandas as pd
from matplotlib.colors import BoundaryNorm
import cmaps
from datetime import timedelta
import matplotlib.dates as mdates

In [ ]:
def process_set(set_name, base_dir, geo_em_base):
    archive_path = os.path.join(base_dir, set_name, f"{set_name}_wrfouts.tar.gz")
    extract_dir = os.path.join(base_dir, set_name, "temp_extract_dir")

    files = listWrfouts(archive_path, extract_dir)
    print(f"Listing wrfouts from {archive_path}")
    domain_files = defaultdict(list)
    for f in files:
        if "d01" in os.path.basename(f):
            domain_files["d01"].append(f)
        elif "d02" in os.path.basename(f):
            domain_files["d02"].append(f)

    result = defaultdict(list)
    print(f"Listed wrfouts from {archive_path}")

    for dom in domain_files:
        domain_files[dom] = sorted(domain_files[dom])
        geo_em_path = os.path.join(geo_em_base, f"geo_em.{dom}.nc")
        for file in domain_files[dom]:
            print(f"Injecting missing values into {file}, from {geo_em_path}")
            inject_missing_coordinates_from_geo_em(file, geo_em_path)
        result[dom] = [Dataset(path, mode="r") for path in domain_files[dom]]

    return set_name, result

# Read WRF outputs 

In [3]:
base_dir = r"D:\istanbul_wrfouts\20012024"
sets = [f"SET{i}" for i in range(1, 16)]
df = {}

for set in sets:
    df[set] = {"d01": [], "d02": []}
    
    wrfouts_list = listWrfouts(os.path.join(base_dir, set, f"{set}_wrfouts.tar.gz"), os.path.join(base_dir, set, "temp_extract_dir"))
    for f in wrfouts_list:
        if "d01" in os.path.basename(f):
            df[set]["d01"].append(Dataset(f))
        elif "d02" in os.path.basename(f):
            df[set]["d02"].append(Dataset(f))
            gc.collect()

In [ ]:
base_dir = r"D:\istanbul_wrfouts\20012024"
geo_em_base = r"D:\istanbul_wrfouts"
sets = [f"SET{i}" for i in range(1, 16)]

df = {}

for set in sets:
    set_name, data = process_set(set, base_dir, geo_em_base)
    df[set_name] = data
    gc.collect()

In [4]:
for i in range(1, 16):
    try:
        set = "SET"+str(i)
        get_point_data(df[set]["d02"], "t2", 41.223333,29.165833)
    except:
        print(f"SET{i}")

# Read CSV files

In [ ]:
sets_mapping = pd.DataFrame([
    {"set_name": "SET1", "physics": "mp4_pbl1"},
    {"set_name": "SET2", "physics": "mp4_pbl5"},
    {"set_name": "SET3", "physics": "mp4_pbl7"},
    {"set_name": "SET4", "physics": "mp6_pbl1"},
    {"set_name": "SET5", "physics": "mp6_pbl5"},
    {"set_name": "SET6", "physics": "mp6_pbl7"},
    {"set_name": "SET7", "physics": "mp38_pbl1"},
    {"set_name": "SET8", "physics": "mp38_pbl5"},
    {"set_name": "SET9", "physics": "mp38_pbl7"},
    {"set_name": "SET10", "physics": "mp10_pbl1"},
    {"set_name": "SET11", "physics": "mp10_pbl5"},
    {"set_name": "SET12", "physics": "mp10_pbl7"},
    {"set_name": "SET13", "physics": "mp24_pbl1"},
    {"set_name": "SET14", "physics": "mp24_pbl5"},
    {"set_name": "SET15", "physics": "mp24_pbl7"},
])

In [ ]:
csv_path = os.path.join(os.getcwd(), "csv_files")
case = "20241123"
station = "17061"

matching_files = []
df = {}

if os.path.isdir(csv_path):
    for filename in os.listdir(csv_path):
        if f"case_{case}" in filename and f"station_{station}" in filename and filename.endswith(".csv"):
            matching_files.append(os.path.join(csv_path, filename))

for file in matching_files:
    physics = f'{os.path.basename(file).split("_")[2]}_{os.path.basename(file).split("_")[3]}'
    set_name = sets_mapping.loc[sets_mapping["physics"] == physics, "set_name"].iloc[0]
    
    df[set_name] = pd.read_csv(file)
